In [2]:
!pip install selenium

  Using cached selenium-4.21.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached trio-0.25.1-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
Using cached selenium-4.21.0-py3-none-any.whl (9.5 MB)
Using cached trio-0.25.1-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Attempt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.18.18 requires urllib3<1.26,>=1.20; python_version != "3.4", but you have urllib3 2.2.1 which is incompatible.
requests 2.18.4 requires urllib3<1.23,>=1.21.1, but you have urllib3 2.2.1 which is incompatible.


In [1]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import csv

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# 변수 설정
QUERY = '쿠팡'
search_QUERY = urlencode({'subquery': QUERY}, encoding='utf-8')
URL = f"https://www.fmkorea.com/search.php?mid=football_korean&search_keyword={QUERY}&search_target=title_content"

def get_posts(page_num):
    global QUERY
    posts = []
    for page in range(page_num + 1):
        # 페이지 넘기기
        board_link = f"https://www.fmkorea.com/search.php?mid=football_korean&search_keyword={QUERY}&search_target=title_content&page={page+1}"
        req = requests.get(board_link)
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'html.parser')
        # 전체 20개의 게시글의 링크를 posts에 append하는 과정
        tds = soup.find_all('td', {'class': 'title hotdeal_var8'})
        for td in tds:
            post = td.find('a', {'class': 'hx'})
            if post is not None:
                posts.append(post['href'])
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다.")
    # 게시글 링크 csv로 저장
    post_file = open(f"FMkorea_{QUERY}_{page_num}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    return posts

def extract_info(url):
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html,'html.parser')
    title = soup.find('span', {'class': 'np_18px_span'}).get_text(strip=True)
    user_id = soup.find_all('a', {'class': lambda x: x and x.startswith('member_')}).get_text(strip=True)
    post_time = soup.find('span', {'class': 'date m_no'}).get_text(strip=True)
    post = soup.find('div', class_='xe_content').get_text(seperator = ' ', strip=True)
    view_cnt = int(soup.find('span', text=lambda x: x and '조회 수' in x).find('b').text)
    recomm_cnt = int(soup.find('span', text=lambda x: x and '추천 수' in x).find('b').text)
    reply_cnt = int(soup.find('b').text)
    reply_content = []#여기서부터 수정!! 댓글수가 많을경우 댓글 페이지가 넘어감. 댓글 50게 넘어가면 페이지 생기는듯
    if reply_cnt != 0:
        replies = soup.find('ul', {'class': 'fdb_lst_ul'})#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        for reply in replies:
            reply_author = reply.find('span', {'class': 'name'}).get_text(strip=True)
            reply_text = reply.find('span', {'class': 're_txt'}).get_text(strip=True).replace('\n', '').replace('\r', '').replace('\t', '')
            reply_content.append({'author': reply_author, 'text': reply_text})

    data = {'title': title, 'user_id': user_id, 'post_time': post_time, 'post': post, 'view_cnt': view_cnt,
            'recomm_cnt': recomm_cnt, 'reply_cnt': reply_cnt, 'reply_content': reply_content}

    # 댓글이 있다면 딕셔너리에 추가
    if reply_cnt != 0:
        for i, reply_info in enumerate(reply_content):
            data[f'reply_{i+1}_author'] = reply_info['author']
            data[f'reply_{i+1}_text'] = reply_info['text']

    print(url, '완료')
    return data

def get_contents(posts):
    for post_link in posts:
        content = extract_info(post_link)
        append_to_file(f"MLBPARK_{QUERY}_0228.csv", content)
    return print("모든 작업이 완료되었습니다.")

def save_to_file():
    file = open(f'MLBPARK_{QUERY}_0228.csv', mode='a', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(['title', 'user_id', 'post_time', 'post', 'view_cnt', 'recomm_cnt', 'reply_cnt', 'reply_author', 'reply_content'])
    file.close()
    return file

def append_to_file(file_name, data):
    file = open(file_name, mode='a', encoding='utf-8', newline='')  # newline 추가
    writer = csv.writer(file)

    # Write main post data to CSV
    main_post_row = [
        data['title'],
        data['user_id'],
        data['post_time'],
        data['post'],
        data['view_cnt'],
        data['recomm_cnt'],
        data['reply_cnt']
    ]

    # Check if there are replies
    if 'reply_content' in data and data['reply_cnt'] != 0:
        # Write main post data along with reply data to CSV
        for i, reply_info in enumerate(data['reply_content']):
            writer.writerow(main_post_row + [
                reply_info.get('author', ''),  # Reply author
                reply_info.get('text', '')  # Reply text
            ])
    else:
        # Write only main post data to CSV
        writer.writerow(main_post_row + ['']*2)

    file.close()
    return

# 크롤링할 페이지 수 설정
PAGES = 50
# 게시글 링크 수집
post_links = get_posts(PAGES)
# 결과를 저장할 CSV 파일 생성
save_to_file()
# 게시글 정보 크롤링 및 저장
get_contents(post_links)